# Requirements

## Hardware
- Tested on debian-like machine (pop-os to be exact), but ubuntu probably works

## Software
- Tested with g++ --version=10.3.0
```
sudo apt-get install g++ // or
sudo apt-get build-essential
```
-  Install **openMP**
```
sudo apt-get install libomp-dev
```
**openMP** is a library that offers support for seamless parallelization through thread launching. As the programmer, you only need to identify parts of your code feeble for parallelization, while the library itself is an abstractions that helps with loop indexing, thread launching, built-in locks, and is portable on different os, making it a nice option for users who don't care about hardware optimations for every different machine. 

For the graphing tools we will need python3 and matplotlib
```
sudo apt-get install python
# In this repository
python3 -m venv venv
source venv/bin/activate
pip install matplotlib
```


# Description

Follow this [youtube link from Geometry Lab](https://www.youtube.com/watch?v=NDGpsfwAaqo&t=705s&ab_channel=GeometryLab) for a more detailed explanation of a skiplist. Here is a summary of the properties of skiplist.

| Insertion | Deletion | Search |
| --------- | -------- | ------ |
| log(n)    | log(n)   | log(n) |

\*expected time

The skiplist is able to achieve these marks because it uses layers/levels of linked list (visually) stacked on top of each other for *skipping* sections of a normal linked list (which has linear time search), and thus tries to emulate the methodology behind *binary search*.


In the next few sections, I analyze 4 different skiplist implementations: [sequential](https://github.com/bustin11/parallel-skiplist/tree/sequential), [coarse-grain](https://github.com/bustin11/parallel-skiplist/tree/coarse-grain), [fine-grain](https://github.com/bustin11/parallel-skiplist/tree/fine-grain), and [lock-free](https://github.com/bustin11/parallel-skiplist/tree/lock-free). If want to run yourself, make sure you follow the Software guidelines and pull all 4 branches.

# Grab Metrics

In [22]:
def run(branch_name):
    !git checkout $branch_name
    !./run.sh
    !make clean

branch_names = ["sequential", "coarse-grain", "fine-grain", "lock-free"]

In [24]:
!rm output.txt
for name in branch_names:
    !echo $name >> output.txt
    run(name)